<a href="https://colab.research.google.com/github/m-bashari-m/sample-DNN-projects/blob/main/3.3_EarlyStopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
!mkdir dataset
!wget https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat -P dataset

--2022-03-03 20:50:55--  https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 929 [text/plain]
Saving to: ‘dataset.py’

dataset.py          100%[===================>]     929  --.-KB/s    in 0s      

2022-03-03 20:50:55 (33.9 MB/s) - ‘dataset.py’ saved [929/929]

--2022-03-03 20:50:55--  https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset/Data_hoda_full.mat 

In [2]:
from tensorflow import keras
from keras import layers
from keras.models import Sequential
import numpy as np
from dataset import load_hoda
import matplotlib.pyplot as plt

In [3]:


np.random.seed(123)  # for reproducibility

# Load pre-shuffled HODA data into train and test sets
x_train_original, y_train_original, x_test_original, y_test_original = load_hoda(
                                                                        training_sample_size=3500,
                                                                        test_sample_size=400,size=28)



In [4]:
# Preprocess input data
''' 3.1: input data in numpy array format'''
x_train = np.array(x_train_original)
x_test = np.array(x_test_original)

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') /255

# Reshape to original image shape (n x 784)  ==> (n x 28 x 28 x 1)
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)


# 4. Preprocess class labels
y_train = keras.utils.to_categorical(y_train_original, num_classes=10)
y_test = keras.utils.to_categorical(y_test_original, num_classes=10)

In [5]:
# test and validation set
x_val = x_test[:200]
x_test = x_test[200:]
y_val = y_test[:200]
y_test = y_test[200:]

In [14]:
# 5. Define model architecture
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))


# 6. Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
class EarlyStoppingByValAcc(keras.callbacks.Callback):
  def __init__(self, stop_acc, **kwargs):
    super(EarlyStoppingByValAcc, self).__init__(**kwargs)
    self.stop_acc = stop_acc

  def on_epoch_end(self, epoch, logs):
    if logs.get('val_accuracy') > self.stop_acc:
      print('\nThe training stopped because accuracy is greater than {}'.format(self.stop_acc))
      self.model.stop_training = True

In [16]:
early_stopping = EarlyStoppingByValAcc(.75)

In [17]:
history = model.fit(x_train, y_train,
                    epochs=200,
                    batch_size=256,
                    validation_data = (x_val, y_val),
                    callbacks=[early_stopping])

Epoch 1/200
14/14 [==============================] - 3s 191ms/step - loss: 2.0833 - accuracy: 0.3097 - val_loss: 1.4491 - val_accuracy: 0.7050
Epoch 2/200
14/14 [==============================] - ETA: 0s - loss: 1.2406 - accuracy: 0.5997
The training stopped because accuracy is greater than 0.75
14/14 [==============================] - 2s 178ms/step - loss: 1.2406 - accuracy: 0.5997 - val_loss: 0.5943 - val_accuracy: 0.8100
